# Image Analysis 2021

This Jupyter notebook is part of the course Image Analysis from Radboud University (Nijmegen, Netherlands), and it was developed by researchers of Radboud University Medical Center (Nijmegen, Netherlands).

You should have obtained this notebook by downloading it from the official Brightspace page of the course.
If this is not the case, you should contact the course coordinator at this email address: geert.litjens@radboudumc.nl

This notebook formulates an assignment as part of the course, and the content of this notebook should be used solely to develop a solution to this assignment.
You should not make the code provided in this notebook, or your own solution, publicly available.

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` by substituting `None` variables or by adding your own solution and any place that says "YOUR ANSWER HERE" with your answers to the questions. Note that it is perfectly fine to substitute the images in the exercises with your own if you want to. Please fill in your name and collaborators below:

## Students
Please fill in this cell with your name and e-mail address. This information will be used to track completion of the assignments.

* Name student #1, email address: ...
* Name student #2, email address (optional): ...

## Instructions

* Groups: You should work in **groups of maximum 2 people**.
* Deadline for this assignment: 
 * Preferably before May 12th
 * Please upload your **fully executed** notebook to BrightSpace
  The file name of the notebook you submit must be ```NameSurname1_NameSurname2.ipynb```

This notebooks contains cells with snippets of code that we provide in order to load and visualize data, but also some convenience functions that could be useful to develop your assignment.

We also provide templates for functions that have to be implemented, with a given list of input variables and some output variables.

Your submission should contain the **fully executed** notebook with **your code** implemented, as well as **your answers** to questions.

## Libraries

First, we import the basic libraries necessary to develop this assignment.

In [ ]:
import skimage as ski # For reading images
import skimage.transform as skit # Basic image transformation functions
import skimage.filters # Functions for image filtering
from   scipy.signal import convolve2d # 2D convolution function
from   scipy.ndimage import gaussian_filter # Filter images with a Gaussian function or derivative
from   skimage.color import label2rgb # Color labels for segmented objects
from   skimage.transform import hough_circle, hough_circle_peaks
from   skimage.draw import circle_perimeter # Draw a circle on an image
import matplotlib.pyplot as plt # Plotting and visalization of images
import numpy as np # Basic math and array functions

# Basic convolution

First we will covert the basics of the convolution operation. Below is a function to plot an image in addition to its pixel values, which makes what a convolution is doing a bit more insightful.

In [ ]:
def imshow_with_values(array, axis):
    axis.axis(False)
    axis.imshow(array, cmap='gray');
    for (j,i),label in np.ndenumerate(array):
        axis.text(i, j, label,ha='center',va='center', fontsize=12, color="black", backgroundcolor="white")

The cell below generates a simple 5x5 image and a 3x3 filter and convolves them. You can modify the filter to see what happens with different values.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(24, 8)) 
test_img = np.arange(1, 26).reshape(5, 5)
test_filter = np.array(([1,1,1], [1,1,1], [1,1,1]), dtype="ubyte")
convolved_test_img = convolve2d(test_img, test_filter, mode='same', boundary='fill')
imshow_with_values(test_img, axes[0]);
imshow_with_values(test_filter, axes[1]);
imshow_with_values(convolved_test_img, axes[2]);

<font color="blue">**Question:** Experiment a bit with different ways convolution can handle the boundaries of the image. What happens when you change `mode` in the `convolve2d` from `same` to `valid`? Or when you change `boundary` from `fill` to `wrap`? Can you explain why these differences happen?

YOUR ANSWER HERE

<font color="blue">**Question:** As you may notice, the values in the image increase substantially after convolution. Why is that? What can you do to mitigate this value increase such that mean of the convolved area stays the same?

YOUR ANSWER HERE

# Noise removal
In this part of the exercise we will look at image filtering for enhancement of image quality. We'll start with a clean color images and we will add two different types of noise to it. The first type is Gaussian noise, which is normally distributed, the second one is salt-and-pepper noise, which switches a pixel on or off.

In [ ]:
rocket = ski.data.rocket()
rocket_gaussian_noise = np.clip(rocket + 16 * np.random.default_rng().normal(0, 1, rocket.shape), 0, 255).astype("ubyte")
rocket_1D = rocket.flatten()
salt_indices = np.random.choice(rocket_1D.shape[0], 25000, replace=False)
pepper_indices = np.random.choice(rocket_1D.shape[0], 25000, replace=False)
rocket_1D[salt_indices] = 255
rocket_1D[pepper_indices] = 0
rocket_salt_pepper_noise = rocket_1D.reshape(rocket.shape)
fig, axes = plt.subplots(1, 3, figsize=(24, 8)) 
axes[0].imshow(rocket); # Show the orignal image
axes[1].imshow(rocket_gaussian_noise); # Show the image with Gaussian noise
axes[2].imshow(rocket_salt_pepper_noise); # Show the image with salt-and-pepper noise

As a first step we will try to remove the noise with smoothing filters, such as averaging filters. First we define a convolution filter for RGB images which applies a filter to each channel:

In [ ]:
def convolve2d_rgb(image, filter, **kwargs):
    return np.array([convolve2d(image[:, :, i], filter, **kwargs) for i in range(3)]).transpose(1,2,0).astype("ubyte")

<font color="blue">**Assignment:** Now we need to define the smoothing filter. We will start with a basic 5x5 averaging filter, replace the `None` below. Then we visualize the results.

In [ ]:
# Replace None with your code
average_filter = None
filtered_rocket_gaussian_noise = convolve2d_rgb(rocket_gaussian_noise, average_filter)

Let's show the result of the filtering process below. You can experiment with different filter sizes if you want.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(24, 8)) 
axes[0].imshow(rocket); # Show the orignal image
axes[1].imshow(rocket_gaussian_noise); # Show the noisy image
axes[2].imshow(filtered_rocket_gaussian_noise); # Show the filtered image

Let's also have a look at one row through the image in a bit more detail to get a feeling how the noise affects the image:

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(24, 8))
axes[0].plot(rocket[250,:,0])
axes[1].plot(rocket_gaussian_noise[250,:,0])
axes[2].plot(filtered_rocket_gaussian_noise[250,:,0])

Let's try to fix the image with the salt-and-pepper noise using the same filter and visualize those results as well:

In [ ]:
filtered_rocket_salt_pepper_noise = convolve2d_rgb(rocket_salt_pepper_noise, average_filter)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(24, 8)) 
axes[0].imshow(rocket); # Show the orignal image
axes[1].imshow(rocket_salt_pepper_noise); # Show the noisy image
axes[2].imshow(filtered_rocket_salt_pepper_noise); # Show the smoothed image

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(24, 8))
axes[0].plot(rocket[250,:,0])
axes[1].plot(rocket_salt_pepper_noise[250,:,0])
axes[2].plot(filtered_rocket_salt_pepper_noise[250,:,0])

As you can see from the result image and the line plot, much noise is still present in the image, especially in the background. We can probably do a better job using a median filter. Below is the skeleton of a median filter function for RGB images. Modify it to apply a median filter to every channel separately. You can use the `convolve2d_rgb` function as an example. To apply a median filter to a single channel, have a look at the `ski.filters.median` function. Remember you can use `?ski.filters.median` to get the documentation. **Note:** the median filter can also use non-square filter shapes, you can use a square filter for simplicity.

In [ ]:
def median_filter_rgb(image, filter_size, **kwargs):
    filtered_image = None
    return filtered_image

In [ ]:
median_filtered_rocket = median_filter_rgb(rocket_salt_pepper_noise, 5)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(24, 8)) 
axes[0].imshow(rocket); # Show the orignal image
axes[1].imshow(rocket_salt_pepper_noise); # Show the noisy image
axes[2].imshow(median_filtered_rocket); # Show the smoothed image

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(24, 8))
axes[0].plot(rocket[250,:,0])
axes[1].plot(rocket_salt_pepper_noise[250,:,0])
axes[2].plot(median_filtered_rocket[250,:,0])

<font color="blue">**Question:** Now try to use the median filter for the image corrupted with Gaussian noise. What do you observe? Is it as effective as a averaging filter? What happens if you increase the value of the center pixel of the averaging filter? (Don't forget to alter the normalization constant). Just provide your observations and reasoning.

YOUR ANSWER HERE

# Edge Detection
Filtering using convolutions can also be used to highlight specific elements in an image, for example edges. Typically you can use a difference operator to highlight the edges. However, this can cause issues in the presence of noise. Let's have a look at some basic edge operators.

### Difference, Prewitt and Sobel operators

First, let's define the different filters. You can flip the filters to a different axis (x or y) by using the `transpose` command.

In [ ]:
difference_x = np.array([[1, 0, -1]])
difference_y = difference_x.transpose()

Now let's see what happens when we apply this to a normal and a noisy image. For ease of visualization we will use grayscale images for the edge detection part.

In [ ]:
rocket_gray = np.mean(rocket, axis=2)
rocket_noise_gray = np.clip(rocket_gray + 8 * np.random.default_rng().normal(0, 1, rocket_gray.shape), 0, 255).astype("ubyte")

In [ ]:
# Normal image
rocket_dx = convolve2d(rocket_gray, difference_x, mode='same')
rocket_dy = convolve2d(rocket_gray, difference_y, mode='same')
rocket_grad_mag = np.sqrt(rocket_dx**2 + rocket_dy**2)

# Noisy image
rocket_noise_dx = convolve2d(rocket_noise_gray, difference_x, mode='same')
rocket_noise_dy = convolve2d(rocket_noise_gray, difference_y, mode='same')
rocket_noise_grad_mag = np.sqrt(rocket_noise_dx**2 + rocket_noise_dy**2)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(24, 8)) 
axes[0].imshow(rocket_grad_mag, cmap = "gray"); # Show the orignal image
axes[1].imshow(rocket_noise_grad_mag, cmap = "gray"); # Show the noisy image
axes[2].imshow(rocket_noise_grad_mag[50:150, 400:500], cmap = "gray")
print("Noise to background ratio: " + str(np.std(rocket_noise_grad_mag[50:100, 400:450]) / np.std(rocket_grad_mag[50:100, 400:450])))

As you can see the regular difference operator for edge detection cannot handle noise very well, it creates a lot of 'wrong' edge pixels in the gradient magnitude image. As you have seen during the lecture, there are other filters which incorporate smoothing, such as the Prewitt or Sobel edge detection filters. Let's have a look at how they do

In [ ]:
prewitt_x = convolve2d(np.array([[1, 1, 1]]).transpose(), difference_x, mode="full")
prewitt_y = convolve2d([[1, 1, 1]], difference_y, mode="full")
print(prewitt_x)
print(prewitt_y)

In [ ]:
# Normal image
rocket_dx = convolve2d(rocket_gray, prewitt_x, mode='same')
rocket_dy = convolve2d(rocket_gray, prewitt_y, mode='same')
rocket_grad_mag = np.sqrt(rocket_dx**2 + rocket_dy**2)

# Noisy image
rocket_noise_dx = convolve2d(rocket_noise_gray, prewitt_x, mode='same')
rocket_noise_dy = convolve2d(rocket_noise_gray, prewitt_y, mode='same')
rocket_noise_grad_mag = np.sqrt(rocket_noise_dx**2 + rocket_noise_dy**2)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(24, 8)) 
axes[0].imshow(rocket_grad_mag, cmap = "gray"); # Show the orignal image
axes[1].imshow(rocket_noise_grad_mag, cmap = "gray"); # Show the noisy image
axes[2].imshow(rocket_noise_grad_mag[50:150, 400:500], cmap = "gray")
print("Noise to background ratio: " + str(np.std(rocket_noise_grad_mag[50:100, 400:450]) / np.std(rocket_grad_mag[50:100, 400:450])))

<font color="blue">**Question:** Can you explain what difference you see between the noisy image convolved with the difference kernel and with the Prewitt kernel? And what causes this difference?

YOUR ANSWER HERE

<font color="blue">**Assignment:** Modify the function generating `prewitt_x` and `prewitt_y` to generate Sobel kernels instead. Conduct the edge detection using the Sobel filters. What is the difference between thet two?

YOUR ANSWER HERE

### Canny Edge Detection

For the next exercise, we will implement the Canny Edge detection algorithm from scratch. It contains these four steps:
1. Smooth the image to remove noise
2. Calculate the gradient, gradient magnitude and angle
3. Non-local maxima suppression to thin edges
4. Hysteris thresholding

The first step is intended to get rid of noise in the image. The second step calculates the gradient magnitude and angle to identify the strength of the edges and the direction. The third step thins the edges so only the strongest edge pixels per edge remains. The last steps removes weak edges which are not attached to a strong edge.

We start by generating the filters we will use for the smoothing and edge detection.

In [ ]:
average_filter = (1/25) * np.ones((5, 5))
prewitt_x = convolve2d(np.array([[1, 1, 1]]).transpose(), difference_x, mode="full")
prewitt_y = convolve2d([[1, 1, 1]], difference_y, mode="full")

The function below performs the non-maxima suppression.

In [ ]:
def non_max_suppression(gradient_magnitude, gradient_angle):
 
    image_row, image_col = gradient_magnitude.shape
 
    output = np.zeros(gradient_magnitude.shape)
    
    # Loop over all pixels in the image
    for row in range(1, image_row - 1):
        for col in range(1, image_col - 1):
            # Get the local edge angle in radians
            angle = gradient_angle[row, col]

            # Check whether it is a vertical or a horizontal angle. The angles range from -pi to pi.
            # Extract the pixels before and after the current one based on the direction of the edge
            if ((-np.pi / 4) <= angle <= (np.pi / 4)) or ((3 * np.pi / 4) <= angle) or ((-3 * np.pi / 4) >=  angle):
                before_pixel = gradient_magnitude[row, col - 1]
                after_pixel = gradient_magnitude[row, col + 1]
            else:
                before_pixel = gradient_magnitude[row + 1, col]
                after_pixel = gradient_magnitude[row - 1, col]

            # If the current pixel is higher than its neigbors, keep it, otherwise becomes 0
            if gradient_magnitude[row, col] >= before_pixel and gradient_magnitude[row, col] >= after_pixel:
                output[row, col] = gradient_magnitude[row, col]
    return output

Now we will conduct the steps in turn:

In [ ]:
# 1. Smooth the image to remove noise
smoothed_rocket = convolve2d(rocket_noise_gray, average_filter, mode="same")
# 2. Get the derivatives in both directions and calculate the magnitude and angle maps
rocket_dx = convolve2d(smoothed_rocket, prewitt_x, mode="same")
rocket_dy = convolve2d(smoothed_rocket, prewitt_y, mode="same")
rocket_grad_mag = np.sqrt(rocket_dx**2 + rocket_dy**2)
rocket_grad_ang = np.arctan2(rocket_dy, rocket_dx)
# 3. Perform the thinning of the edges
rocket_thinned = non_max_suppression(rocket_grad_mag, rocket_grad_ang)
# 4. Remove weak edges not connceted to strong edges
rocket_edge = ski.filters.apply_hysteresis_threshold(rocket_thinned, np.percentile(rocket_thinned, 80), np.percentile(rocket_thinned, 97))

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(32, 12)) 
axes[0][0].imshow(label2rgb(rocket_edge, rocket_noise_gray, colors=[(0,1,0)], alpha=0.5, bg_label=0, bg_color=None, kind="overlay")) # Original grayscale image with edges overlayed in green
axes[0][1].imshow(smoothed_rocket, cmap = "gray"); # Smoothed image
axes[0][2].imshow(rocket_grad_mag, cmap = "gray"); # Gradient magnitude
axes[1][0].imshow(rocket_grad_ang, cmap = "gray"); # Gradient angle
axes[1][1].imshow(rocket_thinned, cmap = "gray"); # Thinned edges
axes[1][2].imshow(rocket_edge, cmap = "gray"); # Final result

<font color="blue">**Assignment:** Below is the same process as above, but now we are going to use Gaussian derivatives to perform the smoothing and edge detection in one step. 

First, here is a cell to visualize the Gaussian function and its derivatives for various scales (sigma). This is simply to allow you to explore how these change.

In [ ]:
sigma = 3.
x = np.linspace(-10, 10, 100)
y = np.linspace(-10, 10, 100)

x, y = np.meshgrid(x, y)
gauss = (1/(2*np.pi*sigma**2) * np.exp(-(x**2/(2*sigma**2) + y**2/(2*sigma**2))))
gauss_dx = (x * np.exp(-((x**2+y**2)/(2*sigma**2)))) / (2*np.pi*sigma**4)
gauss_dy = (y * np.exp(-((x**2+y**2)/(2*sigma**2)))) / (2*np.pi*sigma**4)
fig, axes = plt.subplots(1, 3, figsize=(24, 8), subplot_kw={'projection':'3d'}) 
axes[0].plot_surface(x, y, gauss, rstride=3, cstride=3, linewidth=1, antialiased=True,
                cmap="viridis");
axes[0].contourf(x, y, gauss, zdir='z', offset=-0.02, cmap="viridis")
axes[0].set_zlim(-0.02,0.005)
axes[0].set_zticks(np.linspace(0,0.015,2))
axes[0].view_init(30, 45)
axes[1].plot_surface(x, y, gauss_dx, rstride=3, cstride=3, linewidth=1, antialiased=True,
                cmap="viridis");
axes[1].contourf(x, y, gauss_dx, zdir='z', offset=-0.01, cmap="viridis")
axes[1].set_zlim(-0.01,0.005)
axes[1].set_zticks(np.linspace(0,0.005,2))
axes[1].view_init(30, 45)
axes[2].plot_surface(x, y, gauss_dy, rstride=3, cstride=3, linewidth=1, antialiased=True,
                cmap="viridis");
axes[2].contourf(x, y, gauss_dy, zdir='z', offset=-0.01, cmap="viridis")
axes[2].set_zlim(-0.01,0.005)
axes[2].set_zticks(np.linspace(0,0.005,2))
axes[2].view_init(30, 45)

The function below you have to modify to create the gradient magnitude using convolutions with filters based on Gaussian derivatives, which can perform smoothing and edge detection simultanously. You have to use the function `gaussian_filter` on `rocket_noise_gray_float`, which filter the image based on a specified `sigma` and the order of the derivative. An order of 0 means the Gaussian function itself, `[1, 0]` means the first-order partial derivative in the x-direction and `[0, 1]` in the y-direction. Remember to use `?gaussian_filter` if you need more explanations.

In [ ]:
# Replace None with your code
rocket_noise_gray_float = rocket_noise_gray.astype(float)
rocket_dx = None
rocket_dy = None
rocket_grad_mag = None
rocket_grad_ang = None
rocket_thinned = non_max_suppression(rocket_grad_mag, rocket_grad_ang)
rocket_edge = ski.filters.apply_hysteresis_threshold(rocket_thinned, np.percentile(rocket_thinned, 5), np.percentile(rocket_thinned, 95))

<font color="blue">**Assignment:** The function below will again plot the end-result and the thinned_edges. Explore with different values of `sigma`. What happens when you increase or decrease the value of `sigma`? *Note:* you might need to alter the threshold values in `apply_hysteresis_threshold` above when you alter `sigma`

YOUR ANSWER HERE

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(24, 8)) 
axes[0].imshow(label2rgb(rocket_edge, rocket_noise_gray, colors=[(0,1,0)], alpha=0.5, bg_label=0, bg_color=None, kind="overlay"))
axes[1].imshow(rocket_thinned, cmap = "gray");

# Object detection

After the edge detection, it makes sense to actually extract the relevant objects from the image. If these objects can be parameterized using a mathematical equation, such as lines or circles, a Hough Transform might be a good choice. In this exercise we will use the Hough Transform for Circles to extract coins.

In [ ]:
coins = ski.data.coins()
plt.imshow(coins, cmap="gray"); # Show the image

<font color="blue">**Assignment:** First, we will extract the boundaries of the coins using the Canny algorithm you developed above. Replace the None below with your code.

In [ ]:
# Replace None with your code
coin_edges = None
plt.imshow(coin_edges);

This should generate pretty decent result, there are some artifacts, but those should not affect the results of our Hough Transform too much. The function below performs a Hough Transform on the image for a predetermined set of radii. It is currently setup to find a single small coin.

In [ ]:
# Detect two radii
hough_radii = np.arange(15, 25, 5)
hough_res = hough_circle(coin_edges, hough_radii)

# Select the most prominent 3 circles
accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, min_xdistance=5, min_ydistance=5, total_num_peaks=1)

The cell below visualizes the detection, the extracted coin and a slice (it is a 3D volume) of the Hough Transform of `coins_edges`.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(24, 8))
coins_rgb = np.stack((coins,)*3, axis=-1)
for center_y, center_x, radius in zip(cy, cx, radii):
    circy, circx = circle_perimeter(center_y, center_x, radius,
                                    shape=coins.shape)
    coins_rgb[circy, circx] = (220, 20, 20)

ax[0].imshow(coins_rgb, cmap="gray");
ax[1].imshow(coins[cy[0]-radii[0]:cy[0]+radii[0], cx[0]-radii[0]:cx[0]+radii[0]], cmap="gray");
ax[2].imshow(hough_res[0], cmap="gray");

<font color="blue">**Assignment:** Modify the function above to extract all coins at once. You need to alter the parameters of both the `hough_circle` function and the `hough_circle_peak` function. Can you explain what you see in the Hough Transform image? What does the `hough_circle_peak` function do?

YOUR ANSWER HERE